In [41]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [42]:
import pandas as pd # import the Pandas Library
import re # import Regular expression library
from datetime import datetime
import nltk

In [31]:
df1 = pd.read_csv("tesla-headlines-sa.csv", encoding='windows-1250', header=None)

In [33]:
df1['Date'][0] = 'Dec. 9'
df1.head() # display the first 5 rows

,Title,Date
0,"Tesla: Nothing Matters, Until Everything Matters",Dec. 9
1,Tesla: The Hype Does Not Reflect Reality,"The European View•Fri, Dec. 6•742 Comments"
2,Tesla Will Likely Achieve Record Deliveries In...,"iBEV•Wed, Dec. 4•340 Comments"
3,Tesla: Automatic Labeling For Computer Vision,"Trent Eady•Tue, Dec. 3•240 Comments"
4,Tesla: Ready To Re-Enter The Fast Lane,"DoctoRx•Tue, Dec. 3•229 Comments"


In [34]:
#Date modified

In [35]:
#Date modified
print(df1['Date'][1]) # display original

match = re.search(r'\w{3}\.\s\d{1,2}', df1['Date'][1])

modifiedDate = match[0] + ", 2019"

print(modifiedDate) # display modified

The European View•Fri, Dec. 6•742 Comments
Dec. 6, 2019


In [27]:
#variation 2

In [26]:
print(df1['Date'][1000]) # display original

match = re.search(r'\w{3}\.\s\d{1,2}\,\s\d{4}', df1['Date'][1000])

print(match[0]) # display modified

Donn Bailey•Mar. 27, 2018•128 Comments
Mar. 27, 2018


In [ ]:
#varaition 4

In [29]:
print(df1['Date'][200]) # display original (without year)
match = re.search(r'\w{3}\s\d{1,2}', df1['Date'][200])
modifiedDate = match[0] + ", 2019"
print(modifiedDate) # display modified

print(df1['Date'][850]) # display original (with year)
match = re.search(r'\w{3}\s\d{1,2}\,\s\d{4}', df1['Date'][850])
print(match[0]) # display modified

Acolyte of Value•Mon, May 13•351 Comments
May 13, 2019
Quoth the Raven•May 17, 2018•123 Comments
May 17, 2018


In [ ]:
#date and time format

In [36]:
newDate1 = datetime.strptime('Dec. 6, 2019', '%b. %d, %Y').date()
print(newDate1)

newDate2 = datetime.strptime('May 17, 2018', '%b %d, %Y').date()
print(newDate2)

2019-12-06
2018-05-17


In [37]:
#cleaned dates: 

In [ ]:
newDateList = [] # create a list to store the cleaned dates

for dateOfArticles in df1['Date']: # loop every row in the "Date" column
    match = re.search(r'\w{3}\.\s\d{1,2}\,\s\d{4}|May\s\d{1,2}\,\s\d{4}|\w{3}\.\s\d{1,2}|May\s\d{1,2}', 
                      dateOfArticles)

    if re.search(r'\w{3}\.\s\d{1,2}\,\s\d{4}|\w{3}\s\d{1,2}\,\s\d{4}',match[0]):
        fulldate = match[0] # don't append year to string
    else:
        fulldate = match[0] + ", 2019" # append year to string
    
    for fmt in ('%b. %d, %Y', '%b %d, %Y'):
        try:
            newDate = datetime.strptime(fulldate, fmt).date()
            break # if format is correct, don't test any other formats
        except ValueError:
            pass
        
    newDateList.append(newDate) # add new date to the list

if(len(newDateList) != df1.shape[0]):
    print("Error: Rows don't match")
else:
    df1['New Date'] = newDateList # add the list to our original dataframe

df1

In [ ]:
#nltk.download('vader_lexicon')


In [43]:
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/anshulkhatri/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
results = []

for headline in df1['Title']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['headline'] = headline # add headlines for viewing
    results.append(pol_score)

results

In [46]:
df1['Score'] = pd.DataFrame(results)['compound']


In [ ]:
df1

In [48]:
# creates a daily score by summing the scores of the individual articles in each day

In [49]:
df2 = df1.groupby(['New Date']).sum() # creates a daily score by summing the scores of the individual articles in each day


In [50]:
dfEodPrice = pd.read_csv("tsla-eod-prices.csv")


In [53]:
type(dfEodPrice['Date'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [52]:
dfEodPrice['Date'] = dfEodPrice['Date'].astype('datetime64[ns]') 


In [ ]:
# only interested in the “Adj Close”
#let’s drop all unwanted rows. 

In [54]:
dfEodPrice2 = dfEodPrice.drop(['Open', 'High','Low','Close','Volume'], axis=1) # drop unwanted rows
dfEodPrice2.set_index('Date', inplace=True) # set Date coloumn as index

In [ ]:
dfEodPrice2

In [ ]:
#To calculate daily returns, we divide today’s prices by yesterday’s.

In [56]:
dfEodPrice2['Returns'] = dfEodPrice2['Adj Close']/dfEodPrice2['Adj Close'].shift(1) - 1 # calculate daily returns


In [57]:
#Step 1: Lagged the sentiment score



In [58]:
df2['Score(1)'] = df2.shift(1)

In [ ]:
df2

In [61]:
#some days there are no news, so checking that as well.

In [62]:
dfEodPrice3 = pd.merge(dfEodPrice2[['Returns']], df2[['Score(1)']], left_index=True, right_index=True, how='left')


In [ ]:
dfEodPrice3

In [64]:
#cleaning NAN with 0

In [67]:
dfReturnsScore = dfEodPrice3

In [68]:
dfReturnsScore.fillna(0, inplace=True) 
# replace NaN with 0 permanently

In [ ]:
dfReturnsScore

In [72]:
#other factors as well whicih affect the tesla stock price. So we remove all those socere on reage of 0.5 - (-0.5)
dfReturnsScore2 = dfReturnsScore

In [73]:
dfReturnsScore2 = dfReturnsScore[(dfReturnsScore['Score(1)'] > 0.5) | (dfReturnsScore['Score(1)'] < -0.5)]


In [ ]:
dfReturnsScore2

In [75]:
#finding coorealtions 

In [76]:
dfReturnsScore2['Returns'].corr(dfReturnsScore2['Score(1)'])


0.044623813315777285